# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 21 2023 12:58PM,259654,19,9290994,"Graystone, LLC",Released
1,Mar 21 2023 12:58PM,259654,19,9290995,"Graystone, LLC",Released
2,Mar 21 2023 12:58PM,259654,19,9290996,"Graystone, LLC",Released
3,Mar 21 2023 12:57PM,259652,10,MSP219785,"Methapharm, Inc.",Released
4,Mar 21 2023 12:57PM,259652,10,MSP219787,"Methapharm, Inc.",Released
5,Mar 21 2023 12:57PM,259652,10,MSP219788,"Methapharm, Inc.",Released
6,Mar 21 2023 12:57PM,259652,10,MSP219780,"Methapharm, Inc.",Released
7,Mar 21 2023 12:57PM,259652,10,MSP219782,"Methapharm, Inc.",Released
8,Mar 21 2023 12:57PM,259652,10,MSP219783,"Methapharm, Inc.",Released
9,Mar 21 2023 12:57PM,259652,10,MSP219784,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,259650,Released,6
28,259651,Released,3
29,259652,Released,7
30,259653,Released,1
31,259654,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259650,NaN,NaN,6.0
259651,NaN,NaN,3.0
259652,NaN,NaN,7.0
259653,NaN,NaN,1.0
259654,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259596,0.0,1.0,0.0
259599,0.0,2.0,0.0
259606,24.0,30.0,19.0
259610,0.0,1.0,1.0
259613,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259596,0,1,0
259599,0,2,0
259606,24,30,19
259610,0,1,1
259613,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259596,0,1,0
1,259599,0,2,0
2,259606,24,30,19
3,259610,0,1,1
4,259613,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259596,,1,
1,259599,,2,
2,259606,24,30,19
3,259610,,1,1
4,259613,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 21 2023 12:58PM,259654,19,"Graystone, LLC"
3,Mar 21 2023 12:57PM,259652,10,"Methapharm, Inc."
10,Mar 21 2023 12:57PM,259653,10,Emerginnova
11,Mar 21 2023 12:55PM,259651,10,"Methapharm, Inc."
14,Mar 21 2023 12:54PM,259650,10,Hush Hush
20,Mar 21 2023 12:44PM,259649,19,ACG North America LLC
21,Mar 21 2023 12:20PM,259634,15,"Brookfield Pharmaceuticals, LLC"
53,Mar 21 2023 12:10PM,259648,16,"SHL Pharma, LLC"
54,Mar 21 2023 12:06PM,259647,19,"GCH Granules USA, Inc."
55,Mar 21 2023 11:50AM,259645,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 21 2023 12:58PM,259654,19,"Graystone, LLC",,,3
1,Mar 21 2023 12:57PM,259652,10,"Methapharm, Inc.",,,7
2,Mar 21 2023 12:57PM,259653,10,Emerginnova,,,1
3,Mar 21 2023 12:55PM,259651,10,"Methapharm, Inc.",,,3
4,Mar 21 2023 12:54PM,259650,10,Hush Hush,,,6
5,Mar 21 2023 12:44PM,259649,19,ACG North America LLC,,,1
6,Mar 21 2023 12:20PM,259634,15,"Brookfield Pharmaceuticals, LLC",,5,27
7,Mar 21 2023 12:10PM,259648,16,"SHL Pharma, LLC",,,1
8,Mar 21 2023 12:06PM,259647,19,"GCH Granules USA, Inc.",,,1
9,Mar 21 2023 11:50AM,259645,10,Eywa Pharma Inc.,,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 21 2023 12:58PM,259654,19,"Graystone, LLC",3,,
1,Mar 21 2023 12:57PM,259652,10,"Methapharm, Inc.",7,,
2,Mar 21 2023 12:57PM,259653,10,Emerginnova,1,,
3,Mar 21 2023 12:55PM,259651,10,"Methapharm, Inc.",3,,
4,Mar 21 2023 12:54PM,259650,10,Hush Hush,6,,
5,Mar 21 2023 12:44PM,259649,19,ACG North America LLC,1,,
6,Mar 21 2023 12:20PM,259634,15,"Brookfield Pharmaceuticals, LLC",27,5,
7,Mar 21 2023 12:10PM,259648,16,"SHL Pharma, LLC",1,,
8,Mar 21 2023 12:06PM,259647,19,"GCH Granules USA, Inc.",1,,
9,Mar 21 2023 11:50AM,259645,10,Eywa Pharma Inc.,5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 21 2023 12:58PM,259654,19,"Graystone, LLC",3,,
1,Mar 21 2023 12:57PM,259652,10,"Methapharm, Inc.",7,,
2,Mar 21 2023 12:57PM,259653,10,Emerginnova,1,,
3,Mar 21 2023 12:55PM,259651,10,"Methapharm, Inc.",3,,
4,Mar 21 2023 12:54PM,259650,10,Hush Hush,6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 21 2023 12:58PM,259654,19,"Graystone, LLC",3.0,NaN,NaN
1,Mar 21 2023 12:57PM,259652,10,"Methapharm, Inc.",7.0,NaN,NaN
2,Mar 21 2023 12:57PM,259653,10,Emerginnova,1.0,NaN,NaN
3,Mar 21 2023 12:55PM,259651,10,"Methapharm, Inc.",3.0,NaN,NaN
4,Mar 21 2023 12:54PM,259650,10,Hush Hush,6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 21 2023 12:58PM,259654,19,"Graystone, LLC",3.0,0.0,0.0
1,Mar 21 2023 12:57PM,259652,10,"Methapharm, Inc.",7.0,0.0,0.0
2,Mar 21 2023 12:57PM,259653,10,Emerginnova,1.0,0.0,0.0
3,Mar 21 2023 12:55PM,259651,10,"Methapharm, Inc.",3.0,0.0,0.0
4,Mar 21 2023 12:54PM,259650,10,Hush Hush,6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3634895,68.0,6.0,0.0
15,519266,39.0,18.0,0.0
16,259648,1.0,0.0,0.0
19,2077052,27.0,32.0,24.0
20,259623,0.0,17.0,15.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3634895,68.0,6.0,0.0
1,15,519266,39.0,18.0,0.0
2,16,259648,1.0,0.0,0.0
3,19,2077052,27.0,32.0,24.0
4,20,259623,0.0,17.0,15.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,68.0,6.0,0.0
1,15,39.0,18.0,0.0
2,16,1.0,0.0,0.0
3,19,27.0,32.0,24.0
4,20,0.0,17.0,15.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,68.0
1,15,Released,39.0
2,16,Released,1.0
3,19,Released,27.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Completed,0.0,0.0,0.0,24.0,15.0
Executing,6.0,18.0,0.0,32.0,17.0
Released,68.0,39.0,1.0,27.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Completed,0.0,0.0,0.0,24.0,15.0
1,Executing,6.0,18.0,0.0,32.0,17.0
2,Released,68.0,39.0,1.0,27.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Completed,0.0,0.0,0.0,24.0,15.0
1,Executing,6.0,18.0,0.0,32.0,17.0
2,Released,68.0,39.0,1.0,27.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()